<h1>Data Cleaning</h1>

<p>This is the second step of the NLP pipeline. For details on the first step (data collection), look under the "scrapers" folder to find the script that scrapes Reuters for company news data based on the stock ticker.</p>

<h2>Scraped Data</h2>

<p>The data that's been scraped comprises</p>
<ul>
    <li>Dates</li>
    <li>Headlines</li>
    <li>First sentences of articles</li>
</ul>

<p>To start off, let's take a look at the raw dataset.</p>

In [1]:
import pandas as pd

In [2]:
# Note that although I'm using AAPL here, the Reuters scraper I wrote has been abstracted to scrape data for any ticker
original_data = pd.read_csv("../dataset/AAPL_news.csv")

In [3]:
original_data

,Date,Headline,Sentence
0,"JULY 23, 2020",U.S. Congressional hearing to question tech gi...,A U.S. congressional hearing scheduled for nex...
1,"JULY 23, 2020",UPDATE 1-U.S. Congressional hearing to questio...,A U.S. congressional hearing scheduled for nex...
2,"JULY 23, 2020",California appeals court rejects Apple TV cons...,A California appeals court has declined a bid ...
3,"JULY 23, 2020",Apple faces deceptive trade practices probe by...,Multiple U.S. states are investigating Apple I...
4,"JULY 23, 2020",Apple faces consumer protection probe by multi...,Multiple U.S. states are investigating Apple I...
...,...,...,...
678,"APRIL 15, 2019",Huawei says not discussed 5G chipsets with App...,China's Huawei Technologies said on Tuesday it...
679,"APRIL 15, 2019","Apple, allies seek billions in U.S. trial test...",Apple Inc and its allies on Monday will kick o...
680,"APRIL 12, 2019",Apple hit with East Texas patent case over iPh...,A New York-based patent monetization company o...
681,"APRIL 12, 2019",Chinese group to get control of Japan Display ...,A Chinese-Taiwanese group will take control of...


<h2>Null values</h2>

<p>A common issue in the date cleaning process is dealing with null values. Let's see how our original dataset fares in this aspect.</p>

In [4]:
original_data.isnull().any()

Date        False
Headline    False
Sentence    False
dtype: bool

<p>As we can see, there aren't any null values in the original dataset which is an indicator that our scraper performed relatively well. But then again, null values are a common issue only when dealing with numeric data, so soon enough, we will have to apply text pre-processing techniques to this dataset.</p>

<h2>Date Format</h2>

<p>Let's convert the "Date" column into a pandas Timestamp (pandas equivalent of the datetime object) and take a look at the results.</p>

In [5]:
df = original_data.copy() # making a copy of the original DataFrame to work with
df["Date"] = pd.to_datetime(df["Date"])

df.head()

,Date,Headline,Sentence
0,2020-07-23,U.S. Congressional hearing to question tech gi...,A U.S. congressional hearing scheduled for nex...
1,2020-07-23,UPDATE 1-U.S. Congressional hearing to questio...,A U.S. congressional hearing scheduled for nex...
2,2020-07-23,California appeals court rejects Apple TV cons...,A California appeals court has declined a bid ...
3,2020-07-23,Apple faces deceptive trade practices probe by...,Multiple U.S. states are investigating Apple I...
4,2020-07-23,Apple faces consumer protection probe by multi...,Multiple U.S. states are investigating Apple I...


<p>That's pretty much everything we have to do to the "Date" column as far as cleaning it up is concerned.</p>

<h2>Common Functions to Clean Text Data</h2>

<p>There are several ways to clean/process text data before feeding it into a model. The exact functions required varies across different use cases. Here are some of the procecssing steps we need to employ for this project</p>

<li>Converting to lowercase</li>

In [6]:
def to_lower(text):
    return text.lower()

<p>For the next set of cleanup functions, we are going to have to make a few imports</p>

In [7]:
import re
import string

<li>Getting rid of punctuation</li>

In [8]:
def remove_punctuation(text):
    return re.sub("[%s]" % re.escape(string.punctuation), "", text)

<li>Replacing special characters</li>

In [9]:
def replace_special_chars(text):
    text = text.replace("â€™", "\'")
    text = text.replace("Ã¤ÃŸ", "")
    text = text.replace("Ã¼", "")
    text = text.replace("  ", " ")
    
    return text

<li>Replacing country names (abbreviations)</li>

In [10]:
def replace_country_names(text):
    text = text.replace("U.S", "United States")
    text = text.replace("EU", "Europe")
    text = text.replace("UK", "United Kingdom")
    
    return text

<li>Removing numbers from words (034220.KS, COVID-19 to COVID etc.)</li>

In [11]:
def remove_numbers(text):
    return re.sub("[\d-]", "", text)

<li>Removing stop words</li>

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')

def remove_stop_words(text):
    text_tokens = word_tokenize(text)
    tokens_no_stop_words = [word for word in text_tokens if not word in stopwords.words()]
    text = " ".join(tokens_no_stop_words)
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<p>This is a good start. As we progress, if we notice unexpected results, we can come back and process the text in a couple more ways by implementing </p>
    <ul>
        <li>Lemmatization - Dealing with word inflections (ex. "take" is the same as "taken" or "took")</li>
        <li>Creating n-grams - Considering n words at a time (ex. a bi-gram would be "United States" as opposed to considering "united" and "states" separately)</li>
        <li>Parts of speech tagging - Recognizing nouns, pronouns (ex. "Canada", "he", "it" etc.)</li>
        <li>Removal of meaningless text</li>
        <li>Removal of stop words - done after discovering the corpus had poor content while doing EDA</li>
        <li>Word embedding - synonyms</li>
    </ul>

<p>For now, let's clean up the text in our dataset using the utility functions defined above.</p>

<h2>Cleaning the Data</h2>

In [13]:
# Applies all the cleanup functions to a piece of text
def clean_up_text(text):
    text = replace_country_names(text)
    text = replace_special_chars(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = to_lower(text)
    text = remove_stop_words(text)
    
    return text


clean = lambda text: clean_up_text(text)

In [14]:
# Creating a DataFrame with clean text
headline_clean = pd.DataFrame(df["Headline"].apply(clean))
sentence_clean = pd.DataFrame(df["Sentence"].apply(clean))
df_clean = pd.concat([headline_clean, sentence_clean], axis=1)
df_clean = pd.concat([df["Date"], df_clean], axis=1)

<h2>Display Text</h2>

<p>Let's make a new feature, "Text", that combines the headline and first sentence for each article. This makes sense as somebody perusing through Reuters will see the headline and first sentence for every article in the listing.</p>

In [15]:
# Creating the "Text" feature that combines headlines and first sentences
df_clean["Headline"] = df_clean["Headline"] + " " # space after last word in headline
df_clean["Text"] = df_clean["Headline"] + df_clean["Sentence"]

<h2>Stock Price</h2>

<p>If you inspect element on the Yahoo Finance page that contains historical stock data (past 5y years), you can find the URL to the dataset that contains the past 5y stock price. To retrieve the data, I'll be using a utility function that I wrote for <a href="https://github.com/anishseeniraj/stockastic">Stockastic</a>.</p>

<img src="../assets/img/yahoo_finance_5y_url.png" alt="Yahoo Finance stock data URL" width=777 height=777>

In [16]:
from datetime import datetime
from datetime import timezone
from datetime import date
from dateutil.relativedelta import relativedelta


def read_historic_data(ticker):
    """
    Reads in historic (past 5y) stock data from Yahoo Finance and returns
    a pandas DataFrame containing the data
    """

    # Calculating today's date and the date 5y ago
    date_today = datetime.today().strftime("%Y-%m-%d")
    dtc = date_today.split("-")
    date_five_years_ago = (
        datetime.today() - relativedelta(years=5)).strftime("%Y-%m-%d")
    dfyc = date_five_years_ago.split("-")

    # Calculates Unix timestamps for start and end dates to fetch data
    timestamp_today = int(datetime(int(dtc[0]), int(dtc[1]), int(
        dtc[2]), 0, 0).replace(tzinfo=timezone.utc).timestamp())
    timestamp_five_years_ago = int((datetime(int(dfyc[0]), int(dfyc[1]), int(
        dfyc[2]), 0, 0)).replace(tzinfo=timezone.utc).timestamp())

    # Reading in stock data from Yahoo Finance in the above timestamps' range
    csv_url = "https://query1.finance.yahoo.com/v7/finance/download/" + ticker + \
        "?period1=" + str(timestamp_five_years_ago) + "&period2=" + \
        str(timestamp_today) + "&interval=1d&events=history"
    df = pd.read_csv(csv_url)

    return df

In [17]:
historic_price = read_historic_data("AAPL")

<p>Let's take a look at the stock price dataset.</p>

In [18]:
historic_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-08-03,121.500000,122.570000,117.519997,118.440002,109.155342,69976000
1,2015-08-04,117.419998,117.699997,113.250000,114.639999,105.653229,124138600
2,2015-08-05,112.949997,117.440002,112.099998,115.400002,106.353653,99312600
3,2015-08-06,115.970001,116.500000,114.120003,115.129997,106.585083,52903000
4,2015-08-07,114.580002,116.250000,114.500000,115.519997,106.946136,38670400


In [19]:
# Changing the date format
historic_price["Date"] = pd.to_datetime(historic_price.Date)

historic_price.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object

In [20]:
# Working with the close prices only
historic_price = historic_price[["Date", "Close"]].copy()

historic_price.head()

,Date,Close
0,2015-08-03,118.440002
1,2015-08-04,114.639999
2,2015-08-05,115.400002
3,2015-08-06,115.129997
4,2015-08-07,115.519997


<p>Let's add a column to the historic price DataFrame indicating whether there was a rise/fall in the stock price in comparison to the previous day. We'll call this feature the <i>inflection</i> of the stock.</p>

In [29]:
historic_price["Inflection"] = 0

for i in range(0, len(historic_price)):
    if(i == 0):
        historic_price["Inflection"][i] = 0
    elif(historic_price["Close"][i] >= historic_price["Close"][i - 1]):
        historic_price["Inflection"][i] = 1
    else:
        historic_price["Inflection"][i] = -1

import warnings

warnings.filterwarnings('ignore')

In [22]:
# Setting Date as the index
historic_price.set_index("Date", inplace=True)
historic_price.head()

,Close,Inflection
Date,,
2015-08-03,118.440002,0
2015-08-04,114.639999,-1
2015-08-05,115.400002,1
2015-08-06,115.129997,-1
2015-08-07,115.519997,1


<p>Now that we've got a feature that indicates whether the stock price went up or down, we can combine this data with the news dataset to move along in our NLP pipeline.</p>

<h2>Adding Stock Price Inflection</h2>

<p>We have to keep in mind that there is the possibility of news articles being released when the market is closed. In this case, I've decided to use the same inflection value as the most recently assigned inflection value.<p>
<p>(<i>Note that this can be dealt with in many ways, but since our sample space is only {-1, 1}, I'm going to stick with using the previous valid inflection value to avoid complications.</i>)</p>

In [23]:
# Setting df_clean's inflection in correspondence with the historic inflection
df_clean["Inflection"] = 0
market_close = 0 # keeps track of the number of articles released during market close

for i in range(0, len(df_clean)):
    try:
        df_clean["Inflection"][i] = historic_price.loc[df_clean["Date"][i]].Inflection
    except:
        market_close += 1
        df_clean["Inflection"][i] = df_clean["Inflection"][i - 1]

print("Number of news articles released when the market was closed -", market_close)

Number of news articles released when the market was closed - 44


In [24]:
df_clean.head()

,Date,Headline,Sentence,Text,Inflection
0,2020-07-23,united states congressional hearing question t...,united states congressional hearing scheduled ...,united states congressional hearing question t...,-1
1,2020-07-23,update united states congressional hearing que...,united states congressional hearing scheduled ...,update united states congressional hearing que...,-1
2,2020-07-23,california appeals court rejects apple tv cons...,california appeals court declined bid consumer...,california appeals court rejects apple tv cons...,-1
3,2020-07-23,apple faces deceptive trade practices probe mu...,multiple united states states investigating ap...,apple faces deceptive trade practices probe mu...,-1
4,2020-07-23,apple faces consumer protection probe multiple...,multiple united states states investigating ap...,apple faces consumer protection probe multiple...,-1


<h2>Duplicates</h2>

<p>We should retain only one entry in the dataset for a given day as opposed to having multiple entries. So we'll keep the most recent article for that day and remove the rest.</p>

In [25]:
df_clean.drop_duplicates(subset=["Date"], inplace=True)
df_clean.reset_index(drop=True, inplace=True)
df_clean.head()

,Date,Headline,Sentence,Text,Inflection
0,2020-07-23,united states congressional hearing question t...,united states congressional hearing scheduled ...,united states congressional hearing question t...,-1
1,2020-07-22,nvidia expresses interest softbanks chip compa...,softbank group corps chip company arm holdings...,nvidia expresses interest softbanks chip compa...,1
2,2020-07-21,gilstrap denies apple bid delay jury trial due...,united states district judge rodney gilstrap m...,gilstrap denies apple bid delay jury trial due...,-1
3,2020-07-15,apple settles east texas patent case drops las...,apple inc agreed settle lawsuit accusing infri...,apple settles east texas patent case drops las...,1
4,2020-07-13,apple says full return offices year bloomberg ...,apple inc aaplo told staff full return united ...,apple says full return offices year bloomberg ...,-1


<h2>Exporting Data</h2>

<p>Now that we've cleaned the dataset a bit, we're almost ready to export it for further use. The two formats that we'll be exporting the data in are<p>

<li>Corpus - Collection of text (what we currently have in df_clean)</li>

In [26]:
df_clean.to_pickle("./pickles/corpus.pkl")

<li>Document-Term Matrix (DTM) - Bag of words (matrix-like structure to keep track of word count)</li>

<p><i>With this method, we'll also be getting rid of stop words (words that add little to no meaning to text).</p>

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# Removing stop words
cv_headline = CountVectorizer(stop_words="english")
cv_sentence = CountVectorizer(stop_words="english")
cv_text = CountVectorizer(stop_words="english")

# Fitting the DTM transformer
headline_cv = cv_headline.fit_transform(df_clean.Headline)
sentence_cv = cv_sentence.fit_transform(df_clean.Sentence)
text_cv = cv_text.fit_transform(df_clean.Text)

# Creating the DTM
headline_dtm = pd.DataFrame(headline_cv.toarray(), columns=cv_headline.get_feature_names())
sentence_dtm = pd.DataFrame(sentence_cv.toarray(), columns=cv_sentence.get_feature_names())
text_dtm = pd.DataFrame(text_cv.toarray(), columns=cv_text.get_feature_names())

# Re-indexing to dates
headline_dtm.index = df_clean.Date
sentence_dtm.index = df_clean.Date
text_dtm.index = df_clean.Date

<p>Let's pickle the DTMs (headlines, sentences, and display text separately).</p>

In [28]:
headline_dtm.to_pickle("./pickles/headline_dtm.pkl")
sentence_dtm.to_pickle("./pickles/sentence_dtm.pkl")
text_dtm.to_pickle("./pickles/text_dtm.pkl")